### **Introducción a LLMs y casos de uso (torchtext + Hugging Face)**

**Objetivos de aprendizaje**

- Comprender qué son los *modelos fundacionales* (Foundation Models) y los *Large Language Models (LLMs)*.
- Cargar y preprocesar texto con **torchtext** para un clasificador sencillo.
- Consumir modelos preentrenados de **Hugging Face Transformers** para tareas prácticas: *zero-shot classification*, *traducción*, *resumen*, *NER* y *text-to-text*.
- Comparar un flujo clásico (dataset+vocab+modelo pequeño) con uno de *foundation models* (pipelines de Transformers).
- Plantear ejercicios interactivos para clase.

> **Nota importante sobre torchtext**: Desde la versión 0.18 (abril 2024) el desarrollo de `torchtext` quedó congelado. Sigue funcionando, pero se recomienda preferir `datasets`/`tokenizers` de Hugging Face para nuevos proyectos. Este cuaderno incluye ambos enfoques para fines didácticos.


#### **1. Instalación de dependencias**

Ejecuta esta celda si estás en un entorno nuevo (por ejemplo, Google Colab). Si ya tienes las librerías, puedes omitirla.


In [ ]:
#!pip -q install --upgrade torchtext==0.18.0 transformers datasets evaluate accelerate sentencepiece sacremoses
!pip install ipywidgets

In [ ]:
import torch, platform
print('PyTorch:', torch.__version__, '| CUDA disponible:', torch.cuda.is_available(), '| Python:', platform.python_version())


#### **2. ¿Qué son los modelos fundacionales (FMs) y los LLMs?**

- **Modelos fundacionales (FMs)**: Modelos entrenados a gran escala con datos amplios (usualmente auto-supervisión) que pueden **adaptarse** a multitud de tareas. Ejemplos: BERT, T5, GPT, CLIP.
- **LLMs**: Subconjunto de FMs centrado en lenguaje natural; suelen usar arquitecturas *Transformer* y tener billones/miles de millones de parámetros. Permiten *in-context learning*, *few-shot*, *zero-shot*.

Ventajas: transferibilidad, cobertura de tareas, rendimiento *SOTA* en muchas benchmarks. Riesgos: sesgos, alucinaciones, coste computacional/energético, actualización de conocimiento.

A continuación, veremos dos flujos de trabajo:

1) **Clásico con torchtext** (dataset->vocab->modelo pequeño): útil para entender el pipeline y entrenar algo rápido.
2) **Con Transformers** (modelos preentrenados + `pipeline`): útil para prototipado veloz y casos de uso reales.


#### **3. Mini-clasificador con `torchtext` (AG_NEWS)**

Entrenaremos un clasificador rápido de noticias con `EmbeddingBag`. Esto ilustra el flujo de preparación de datos, vocabulario y *mini-modelo*.

**Advertencias**:
- Para agilizar la práctica, usaremos un subconjunto pequeño del *train*.
- Si tu versión de `torchtext` difiere, revisa la [documentación] y adapta los imports.


In [ ]:
# (Opcional) Silenciar el warning de deprecación de torchtext 0.18
import warnings
import torchtext
warnings.filterwarnings("ignore", category=UserWarning, module=r"torchtext(\.|$)")

from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

# Carga del dataset
train_iter = AG_NEWS(split='train')
test_iter = AG_NEWS(split='test')

# Construye vocabulario a partir del train
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Re-carga el iterador porque se consumió al construir el vocab
train_iter = AG_NEWS(split='train')

text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1  # etiquetas 1..4 -> 0..3

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (label, text) in batch:
        label_list.append(label_pipeline(label))
        processed_text = torch.tensor(text_pipeline(text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list, text_list, offsets

class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_class = 4
vocab_size = len(vocab)
emsize = 64
modelo = TextClassifier(vocab_size, emsize, num_class).to(device)

def train_func(dataloader):
    modelo.train()
    total_acc, total_count = 0, 0
    log_interval = 200
    optimizer = optim.SGD(modelo.parameters(), lr=4.0)
    scheduler = optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)
    loss_fn = nn.CrossEntropyLoss()
    for epoch in range(1):
        for idx, (label, text, offsets) in enumerate(dataloader):
            optimizer.zero_grad()
            label, text, offsets = label.to(device), text.to(device), offsets.to(device)
            pred = modelo(text, offsets)
            loss = loss_fn(pred, label)
            loss.backward()
            optimizer.step()
            total_acc += (pred.argmax(1) == label).sum().item()
            total_count += label.size(0)
            if idx % log_interval == 0 and idx > 0:
                print(f'  paso {idx:>4d} | acc acumulada: {total_acc/total_count:.3f}')
        scheduler.step()

def test_func(dataloader):
    modelo.eval()
    total_acc, total_count = 0, 0
    with torch.no_grad():
        for label, text, offsets in dataloader:
            label, text, offsets = label.to(device), text.to(device), offsets.to(device)
            pred = modelo(text, offsets)  # <-- corregido (antes: model)
            total_acc += (pred.argmax(1) == label).sum().item()
            total_count += label.size(0)
    print(f'Acc. test: {total_acc/total_count:.3f}')

# Subconjuntos pequeños para una demo rápida
from itertools import islice
train_subset = list(islice(AG_NEWS(split='train'), 0, 2000))
test_subset  = list(islice(AG_NEWS(split='test'),  0, 1000))

train_dataloader = DataLoader(train_subset, batch_size=64, shuffle=True, collate_fn=collate_batch)
test_dataloader  = DataLoader(test_subset,  batch_size=64, shuffle=False, collate_fn=collate_batch)

print('Entrenando...')
train_func(train_dataloader)
print('Evaluando...')
test_func(test_dataloader)


#### **4. Casos de uso con *Foundation Models* vía Hugging Face `pipeline`**

La API `pipeline` ofrece acceso inmediato a tareas comunes sin escribir *boilerplate*.

**4.1 Zero-shot classification**

Permite clasificar texto en etiquetas que **no** se vieron en entrenamiento.


In [ ]:
from transformers import pipeline

zero_shot = pipeline('zero-shot-classification', model='facebook/bart-large-mnli', device_map='auto' if torch.cuda.is_available() else None)
texto = 'Este proyecto integra CI/CD con despliegues canary y observabilidad con métricas y logs estructurados.'
etiquetas = ['DevOps', 'Ciberseguridad', 'Análisis Financiero', 'Educación']
print(zero_shot(texto, candidate_labels=etiquetas))


**4.2 Traducción**



In [ ]:
from transformers import pipeline
import torch

t2t = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",   # tiene safetensors
    device_map="auto" if torch.cuda.is_available() else None
)

def es_en(txt):
    return t2t(f"translate Spanish to English: {txt}", max_new_tokens=128)[0]["generated_text"]

def en_es(txt):
    return t2t(f"translate English to Spanish: {txt}", max_new_tokens=128)[0]["generated_text"]

print(es_en("La ingeniería de IA combina ciencia de datos, MLOps y diseño de productos."))
print(en_es("Large Language Models are transforming software development workflows."))


**4.3 Resumen automático (*summarization*)**


In [ ]:
from transformers import pipeline
import torch

t2t = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",           # suele venir en safetensors
    device_map="auto" if torch.cuda.is_available() else None
)

def resumir_es(texto, max_new_tokens=80):
    prompt = f"Resumir en español en 1-2 frases: {texto}"
    return t2t(prompt, max_new_tokens=max_new_tokens, do_sample=False)[0]["generated_text"]

texto_largo = (
    "Los modelos de lenguaje grandes (LLMs) han habilitado nuevas capacidades en tareas de texto, "
    "como resumen, respuesta a preguntas y generación de código. En entornos educativos, permiten "
    "prototipos rápidos y evaluación automatizada, aunque plantean retos de honestidad académica, "
    "sesgos y trazabilidad. La integración con MLOps y DevSecOps es clave para llevarlos a "
    "producción de forma segura y escalable."
)
print(resumir_es(texto_largo, max_new_tokens=60))



**4.4 NER (Reconocimiento de entidades)**


In [ ]:
# (antes de importar transformers/pipeline)
import os
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"  # quita barras de descarga del Hub
os.environ["DISABLE_TQDM"] = "1"                  # silencia tqdm en transformers

from transformers import pipeline
from transformers.utils import logging as hf_logging
import torch

# baja el ruido de logs de transformers
hf_logging.set_verbosity_error()

# usa la tarea adecuada y la API nueva (sin grouped_entities)
ner = pipeline(
    "token-classification",
    model="dslim/bert-base-NER",
    aggregation_strategy="simple",  # reemplaza grouped_entities=True
    device_map="auto" if torch.cuda.is_available() else None
)

texto = "AWS Lambda se integra con Amazon API Gateway y DynamoDB para arquitecturas serverless."
print(ner(texto))


**4.5 Text-to-text con FLAN-T5 (instrucciones)**

Ideal para plantillas tipo *prompt*: QA, explicación, reescritura, etc.


In [ ]:
instruct = pipeline('text2text-generation', model='google/flan-t5-small', device_map='auto' if torch.cuda.is_available() else None)
print(instruct('Explica brevemente qué es DevSecOps y por qué es importante.')[0]['generated_text'])


### **5. Un vistazo a *tokenization* y *AutoModel***
Para un mayor control, usa `AutoTokenizer`/`AutoModel`.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_id = 'distilbert-base-uncased-finetuned-sst-2-english'
tok = AutoTokenizer.from_pretrained(model_id)
mdl = AutoModelForSequenceClassification.from_pretrained(model_id)

inputs = tok('I love well-instrumented CI pipelines!', return_tensors='pt')
with torch.no_grad():
    outputs = mdl(**inputs)
pred = outputs.logits.argmax(dim=-1).item()
print('Predicción (0=NEG, 1=POS):', pred)


#### **Ejercicios** 

**1) Clasificador torchtext (mini-experimento)**

**Objetivo:** subir el tamaño a \~10 k ejemplos y probar `EmbeddingBag` con `emsize=128`.

**Qué hacer (sin detalles técnicos):**

1. Entrena el clasificador con **10 k** ejemplos de entrenamiento.
2. Prueba **3** tamaños de *batch*: **32**, **64**, **128** (con `emsize=128` fijo).
3. Registra *accuracy* en test y tiempo de entrenamiento.

**Entrega (pega esta tablita con tus números):**

| batch\_size | accuracy\_test | tiempo\_train (s) | notas |
| ----------: | -------------: | ----------------: | ----- |
|          32 |                |                   |       |
|          64 |                |                   |       |
|         128 |                |                   |       |


**Pregunta guía:** ¿Cuál *batch\_size* te dio mejor equilibrio entre estabilidad y tiempo?

**2) Tokenización HF (subwords en español)**

**Objetivo:** comparar cómo trocean palabras dos tokenizadores multilingües.
**Modelos:** `bert-base-multilingual-cased` vs `xlm-roberta-base`.
**Qué hacer:**

1. Toma **un mismo texto corto en español** (2-3 oraciones).
2. Tokeniza con ambos modelos.
3. Cuenta cuántos **subwords** genera cada uno e identifica **5 palabras** donde difieran.

**Entrega:**

| Palabra/frase              | BERT-m-cased (subwords) | XLM-R-base (subwords) | Observación |
| -------------------------- | ----------------------- | --------------------- | ----------- |
| «arquitecturas serverless» |                         |                       |             |
| «observabilidad»           |                         |                       |             |
| …                          |                         |                       |             |

**Pregunta guía:** ¿Cuál produce menos fragmentación en español técnico?


**3) Zero-shot para moderación (conteo manual)**

**Objetivo:** evaluar rápidamente un clasificador *zero-shot* con etiquetas de moderación.
**Qué hacer:**

1. Define **4 etiquetas**: por ejemplo **{"permitido", "spam", "odio", "sensibles"}**.
2. Crea un **set de 20 textos** cortos (inventados o de clase) y márcalos manualmente con la etiqueta "verdadera".
3. Pasa los 20 por el *zero-shot* y guarda la **etiqueta predicha**.
4. Calcula **precisión aproximada**: TP/(TP+FP) para una etiqueta clave (p. ej., "spam").

**Entrega (resumen):**

| etiqueta | TP | FP | precisión ≈ TP/(TP+FP) | comentario |
| -------- | -: | -: | ---------------------: | ---------- |
| spam     |    |    |                        |            |

**Pregunta guía:** ¿En qué tipo de texto falla más (ironía, jerga, abreviaturas)?


**4) Traducción ES<->EN (errores típicos)**

**Objetivo:** probar ida y vuelta en **5 oraciones técnicas** (DevOps/DevSecOps).
**Qué hacer:**

1. Traduce **ES->EN** y **EN->ES** las 5 oraciones.
2. Señala **falsos cognados**, **términos mal traducidos** y **orden extraño**.
3. Da una **corrección humana** breve para los fallos.

**Entrega (por oración):**

* **Original (ES):**
* **Traducción (EN):**
* **Errores detectados:**
* **Corrección sugerida:**

Repite para EN->ES.

**5) Resumen automático (comparación ligera)**

**Objetivo:** comparar dos *summarizers* si el entorno lo permite.
**Modelos sugeridos:** `distilbart-cnn-12-6` vs `facebook/bart-large-cnn` (**si tu máquina los soporta**).
**Si no cargan modelos grandes:** usa un **modelo text-to-text** (por ejemplo., FLAN-T5) como *fallback* y documenta la sustitución.

**Qué hacer:**

1. Elige un **texto fuente** (200-400 palabras).
2. Genera dos resúmenes (o uno + *fallback*).
3. Evalúa con **ROUGE** (si puedes) o, si no, usa una rúbrica simple:

   * **Cobertura** (0-2): ¿incluye las ideas principales?
   * **Coherencia** (0-2): ¿fluye y no repite?
   * **Fidelidad** (0-2): ¿no inventa datos?

**Entrega:**

| sistema             | cobertura (0-2) | coherencia (0-2) | fidelidad (0-2) | notas |
| ------------------- | --------------: | ---------------: | --------------: | ----- |
| Modelo A            |                 |                  |                 |       |
| Modelo B / Fallback |                 |                  |                 |       |

**6) Prompting con FLAN-T5 (tres usos)**

**Objetivo:** practicar instrucciones claras.
\*\*Qué hacer con **un mismo párrafo técnico**:

1. **Clasificación**: "¿A qué tema pertenece? {DevOps, Seguridad, Observabilidad}".
2. **Extracción de claves**: "Devuelve 5 términos clave en español".
3. **Reescritura**: "Reescribe en **estilo académico breve** en 2-3 frases".

**Entrega (para cada prompt):**

* **Entrada:** (párrafo)
* **Salida:** (resultado del modelo)
* **Breve nota:** ¿qué cambiarías en la instrucción para mejorar claridad?

**7) Ética y riesgos**

**Objetivo:** conectar con prácticas DevSecOps.
**Qué cubrir (8-12 líneas):**

* **Sesgos** (ejemplos y mitigaciones básicas: *prompting*, filtros, revisión humana).
* **Privacidad** (datos personales, minimización, anonimización).
* **Trazabilidad** (logs estructurados, *model cards*, versiones de modelo/datos).
* **Despliegue seguro** (control de acceso, tests de *jailbreak*, monitoreo de salida).
* **Ciclo de vida** (actualizaciones del modelo, *rollback*, auditoría).

**Entrega:** un párrafo único, conciso, con 1-2 recomendaciones accionables para tu contexto de curso.



In [ ]:
## Tus respuestas